Required packets

In [1]:
import torch
import pandas as pd
import numpy as np

Compare the original dataset with the tensor obtained after peek detection to find the intervall where all seasonality are present

In [3]:
series = pd.read_csv('../dataset/ARPA/ARPA.csv', header=0, index_col=0, parse_dates=True)

column_names = series.columns.to_list()
time_index = series.index
wavelet_result = torch.load("../Multivariate/result.pt")

def train_test_split(lst, percentage=80.0):
    # Calcola il numero di elementi da prendere
    num_elements = int(len(lst) * (percentage / 100.0))
    # Prendi il numero desiderato di elementi
    return lst[:num_elements]

rows_to_add = []

if len(column_names) > 1:
  # Iter through each row of the DataFrame
  for index, row in series.iterrows():
    tmp_row = [0] * len(row)
    row_index = series.index.get_loc(index)

    for i in range(0, len(column_names)):
      if torch.any(torch.eq(torch.tensor(1), wavelet_result[row_index, i])):
        tmp_row[i] = row[i]
      # if torch.sum(torch.eq(wavelet_result[row_index, i], 1)) >= 2:
      #   tmp_row[i] = row[i]
      else:
        tmp_row[i] = np.nan
    rows_to_add.append(tmp_row)
else:
  for index, row in series.iterrows():
    tmp_row = [0] * len(row)
    row_index = series.index.get_loc(index)
    if torch.any(torch.eq(torch.tensor(1), wavelet_result[row_index])):
      tmp_row = row.values[0]
    # if torch.sum(torch.eq(wavelet_result[row_index], 1)) >= 2:
    #   tmp_row = row.values[0]
    else:
      tmp_row = np.nan
    rows_to_add.append(tmp_row)

# Used to select the percentage of the dataset to be used

# rows_to_add = train_test_split(rows_to_add)
# time_index = train_test_split(time_index)

# Create a DataFrame for the coefficients of the current dimension
final_df = pd.DataFrame(data=rows_to_add, index=time_index, columns=[f'{name}' for name in column_names])

final_df.to_csv('final_dataset.csv')

/tmp/ipykernel_3356/819635723.py:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  series = pd.read_csv('../dataset/ARPA/ARPA.csv', header=0, index_col=0, parse_dates=True)
/tmp/ipykernel_3356/819635723.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  tmp_row[i] = row[i]


## First try move the series

Idea is to move data to obtain a new series with the removed part, due to seasonality match found only in small fraction data are selected based on a single match instead of all

In [17]:
def first_index(data):
    # Trova gli indici dei valori non NaN nel dataset
    indices = np.where(~np.isnan(data))[0]
    
    # Se non ci sono valori non NaN, restituisci -1
    if len(indices) == 0:
        return -1
    
    # Calcola le differenze tra gli indici per trovare i blocchi consecutivi
    diff_indices = np.diff(indices)
    
    # Trova il blocco più lungo di indici consecutivi
    longest_block_index = np.argmax(np.bincount(diff_indices))
    
    # Trova il primo indice di questo blocco nel dataset originale
    first_index_longest_block = indices[np.where(diff_indices == longest_block_index)[0][0]]
    
    return first_index_longest_block

series = pd.read_csv('final_dataset.csv', header=0, index_col=0, parse_dates=True)
time_index = series.index


for column in series.columns:
    col = series[column].values
    max_index = first_index(col)
    print(max_index)
    count = sum(1 for j in range(0, max_index) if not np.isnan(col[j]))
    data_without_nan = [x for x in col if not np.isnan(x)]
    start = max_index-count
    end = len(data_without_nan)-start
    time = time_index[start:end]
    final_df = pd.DataFrame(data = data_without_nan, index=time, columns = [column])
    final_df.to_csv(f'{column}.csv')   



# Create a DataFrame for the coefficients of the current dimension
#final_df = pd.DataFrame(data=data_without_nan, index=time_index, columns=[f'{name}' for name in column_names])

0
0


## Second try use the original data

In [40]:
def first_index(data):
    # Trova gli indici dei valori non NaN nel dataset
    indices = np.where(~np.isnan(data))[0]
    
    # Se non ci sono valori non NaN, restituisci -1
    if len(indices) == 0:
        return -1
    
    # Calcola le differenze tra gli indici per trovare i blocchi consecutivi
    diff_indices = np.diff(indices)
    
    # Trova il blocco più lungo di indici consecutivi
    longest_block_index = np.argmax(np.bincount(diff_indices))
    
    # Trova il primo indice di questo blocco nel dataset originale
    first_index_longest_block = indices[np.where(diff_indices == longest_block_index)[0][0]]
    
    return first_index_longest_block

series = pd.read_csv('final_dataset.csv', header=0, index_col=0, parse_dates=True)
time_index = series.index

for column in series.columns:
    col = series[column].values
    
    max_index = first_index(col)
    count = 0
    start = max_index
    while not np.isnan(col[max_index]) and max_index < len(col)-1:
        count += 1
        max_index += 1
        

    print(start)
    print(count)
    end = start + count
    print(end)
    time = time_index[start:end]
    final_df = pd.DataFrame(data = col[start:end], index=time, columns = [column])
    final_df.to_csv(f'{column}.csv')   


0
80
80
0
85
85
